In [6]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from transformers import BertTokenizer, BertModel
import torch
import random
import esm  # Install ESM: pip install fair-esm

# Load protein interaction network data
data = pd.read_csv('/users/barmanjy/Desktop/Persister Cell/GSE1000_series_matrix.txt', delimiter='\t', skiprows=67, header=None)

# Preprocess your data and construct a graph
def preprocess_data(data):
    # Convert non-numeric values to NaN
    data = data.apply(pd.to_numeric, errors='coerce')
    data = data.dropna(axis=1, how='all')  # Drop columns with all NaNs

    # Calculate correlation matrix
    correlation_matrix = data.corr()

    # Threshold correlations (optional)
    threshold = 0.5
    adjacency_matrix = np.where(abs(correlation_matrix) > threshold, 1, 0)
    return adjacency_matrix

adjacency_matrix = preprocess_data(data)
G = nx.from_numpy_array(adjacency_matrix)

# Custom Transformer to extract network features
class NetworkFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, G):
        features = []
        betweenness = nx.betweenness_centrality(G)
        eigenvector = nx.eigenvector_centrality(G)
        for node in G.nodes():
            node_features = [
                G.degree(node),
                nx.clustering(G, node),
                betweenness[node],
                eigenvector[node]
            ]
            features.append(node_features)
        return np.array(features)

# Function to train deep learning model for feature extraction
def train_deep_learning_model(X_train, y_train, param_grid):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    model = Sequential()
    model.add(LSTM(units=param_grid['units'], input_shape=(X_train_scaled.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer=param_grid['optimizer'], loss='mean_squared_error')

    X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
    model.fit(X_train_reshaped, y_train, epochs=param_grid['epochs'], batch_size=param_grid['batch_size'])
    
    return model, scaler

# Function to calculate PI scores using Ricci curvature
def calculate_pi(G, d, X_network, X_bert, model, scaler):
    X_combined = np.concatenate((X_network, X_bert), axis=1)
    X_combined_scaled = scaler.transform(X_combined)
    X_combined_reshaped = np.reshape(X_combined_scaled, (X_combined_scaled.shape[0], X_combined_scaled.shape[1], 1))
    pi_scores = model.predict(X_combined_reshaped)
    return pi_scores.flatten()

# Function to calculate distances based on edge weights
def calculate_distances(G, a, x):
    distances = {}
    for edge in G.edges():
        i, j = edge
        distances[edge] = a[i][j] * x[i] * x[j]
    return distances

# Function to calculate Ricci curvature
def calculate_ricci_curvature(G, a, x):
    ricci_curvature = {}
    for edge in G.edges():
        i, j = edge
        ricci_curvature[edge] = 2 * (a[i][j] - (x[i] + x[j]))
    return ricci_curvature

# Function to update distances in discrete Ricci flow
def update_distances(G, d, ricci_curvature, delta_t):
    updated_distances = {edge: distance + ricci_curvature[edge] * delta_t for edge, distance in d.items()}
    return updated_distances

# Function to perform discrete Ricci flow iterations
def discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations, convergence_threshold=1e-6):
    d_stem = calculate_distances(G, a, x_stem)
    d_diff = calculate_distances(G, a, x_diff)
    ricci_curvature_stem = calculate_ricci_curvature(G, a, x_stem)
    ricci_curvature_diff = calculate_ricci_curvature(G, a, x_diff)
    prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    for _ in range(num_iterations):
        d_stem = update_distances(G, d_stem, ricci_curvature_stem, delta_t)
        d_diff = update_distances(G, d_diff, ricci_curvature_diff, delta_t)
        # Convergence check
        if np.allclose(np.array(list(d_stem.values())), np.array(list(prev_d_stem.values())), atol=convergence_threshold) and \
           np.allclose(np.array(list(d_diff.values())), np.array(list(prev_d_diff.values())), atol=convergence_threshold):
            break
        prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    return d_stem, d_diff

# Function to extract BERT features using ESM model
def extract_bert_features(sequences):
    model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
    model = model.eval()  # Set the model to evaluation mode
    batch_converter = alphabet.get_batch_converter()

    # Prepare data (here we assume sequences_data is a list of protein sequences)
    data = [(str(i), seq) for i, seq in enumerate(sequences)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    
    token_representations = results["representations"][33]
    sequence_representations = token_representations.mean(1).numpy()
    return sequence_representations

# Load and preprocess biological sequences data
# Assuming sequences_data is a list of protein sequences corresponding to the nodes in the graph
sequences_data = ["sequence1", "sequence2", ...]  # Replace with actual sequences
X_bert = extract_bert_features(sequences_data)

# Placeholder data for training and testing
target_variable = np.random.rand(len(G.nodes()))  # Placeholder for target variable

# Define hyperparameters for the deep learning model
param_grid = {
    'units': 50,
    'optimizer': 'adam',
    'epochs': 10,
    'batch_size': 32
}

print("Shape of adjacency matrix:", adjacency_matrix.shape)
print("Shape of target variable:", target_variable.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(adjacency_matrix, target_variable, test_size=0.2, random_state=42)

# Train deep learning model for feature extraction
model, scaler = train_deep_learning_model(X_train, y_train, param_grid)

# Extract network features
network_feature_extractor = NetworkFeatureExtractor()
X_network = network_feature_extractor.transform(G)

# Placeholder for edge weights based on gene expression data
a = np.random.rand(len(G.nodes()), len(G.nodes()))

# Define initial and final states for the Ricci flow
x_stem = np.random.rand(len(G.nodes()))  # Placeholder for undifferentiated cell state
x_diff = np.random.rand(len(G.nodes()))  # Placeholder for fully differentiated cell state

# Perform discrete Ricci flow iterations
delta_t = 0.1  # Placeholder for time step
num_iterations = 10  # Placeholder for number of iterations
d_stem, d_diff = discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations)

# Calculate proliferation index
pi_scores_stem = calculate_pi(G, d_stem, X_network, X_bert, model, scaler)
pi_scores_diff = calculate_pi(G, d_diff, X_network, X_bert, model, scaler)

# Convert pi_scores_stem and pi_scores_diff to dictionaries for easier manipulation
pi_scores_stem = {node: score for node, score in enumerate(pi_scores_stem)}
pi_scores_diff = {node: score for node, score in enumerate(pi_scores_diff)}

# Categorize cells into groups based on proliferation index
threshold1 = 0.5
threshold2 = 0.3
threshold3 = 0.1

group1 = {node: score for node, score in pi_scores_stem.items() if score > threshold1}
group2 = {node: score for node, score in pi_scores_stem.items() if threshold2 < score <= threshold1}
group3 = {node: score for node, score in pi_scores_stem.items() if threshold3 < score <= threshold2}

# Print the groups
print("Group 1:", group1)
print("Group 2:", group2)
print("Group 3:", group3)


KeyError: 'sequence1'

In [2]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 161.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 51.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 6.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.4 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import esm  # Install ESM: pip install fair-esm
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Load protein interaction network data
data = pd.read_csv('/users/barmanjy/Desktop/Persister Cell/GSE1000_series_matrix.txt', delimiter='\t', skiprows=67, header=None)

# Preprocess your data and construct a graph
def preprocess_data(data):
    # Convert non-numeric values to NaN
    data = data.apply(pd.to_numeric, errors='coerce')
    data = data.dropna(axis=1, how='all')  # Drop columns with all NaNs

    # Calculate correlation matrix
    correlation_matrix = data.corr()

    # Threshold correlations (optional)
    threshold = 0.5
    adjacency_matrix = np.where(abs(correlation_matrix) > threshold, 1, 0)
    return adjacency_matrix

adjacency_matrix = preprocess_data(data)
G = nx.from_numpy_array(adjacency_matrix)

# Custom Transformer to extract network features
class NetworkFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, G):
        features = []
        betweenness = nx.betweenness_centrality(G)
        eigenvector = nx.eigenvector_centrality(G)
        for node in G.nodes():
            node_features = [
                G.degree(node),
                nx.clustering(G, node),
                betweenness[node],
                eigenvector[node]
            ]
            features.append(node_features)
        return np.array(features)

# Function to train deep learning model for feature extraction
def train_deep_learning_model(X_train, y_train, param_grid):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    model = Sequential()
    model.add(LSTM(units=param_grid['units'], input_shape=(X_train_scaled.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer=param_grid['optimizer'], loss='mean_squared_error')

    X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
    model.fit(X_train_reshaped, y_train, epochs=param_grid['epochs'], batch_size=param_grid['batch_size'])
    
    return model, scaler

# Function to calculate PI scores using Ricci curvature
def calculate_pi(G, d, X_network, X_esm, model, scaler):
    X_combined = np.concatenate((X_network, X_esm), axis=1)
    X_combined_scaled = scaler.transform(X_combined)
    X_combined_reshaped = np.reshape(X_combined_scaled, (X_combined_scaled.shape[0], X_combined_scaled.shape[1], 1))
    pi_scores = model.predict(X_combined_reshaped)
    return pi_scores.flatten()

# Function to calculate distances based on edge weights
def calculate_distances(G, a, x):
    distances = {}
    for edge in G.edges():
        i, j = edge
        distances[edge] = a[i][j] * x[i] * x[j]
    return distances

# Function to calculate Ricci curvature
def calculate_ricci_curvature(G, a, x):
    ricci_curvature = {}
    for edge in G.edges():
        i, j = edge
        ricci_curvature[edge] = 2 * (a[i][j] - (x[i] + x[j]))
    return ricci_curvature

# Function to update distances in discrete Ricci flow
def update_distances(G, d, ricci_curvature, delta_t):
    updated_distances = {edge: distance + ricci_curvature[edge] * delta_t for edge, distance in d.items()}
    return updated_distances

# Function to perform discrete Ricci flow iterations
def discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations, convergence_threshold=1e-6):
    d_stem = calculate_distances(G, a, x_stem)
    d_diff = calculate_distances(G, a, x_diff)
    ricci_curvature_stem = calculate_ricci_curvature(G, a, x_stem)
    ricci_curvature_diff = calculate_ricci_curvature(G, a, x_diff)
    prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    for _ in range(num_iterations):
        d_stem = update_distances(G, d_stem, ricci_curvature_stem, delta_t)
        d_diff = update_distances(G, d_diff, ricci_curvature_diff, delta_t)
        # Convergence check
        if np.allclose(np.array(list(d_stem.values())), np.array(list(prev_d_stem.values())), atol=convergence_threshold) and \
           np.allclose(np.array(list(d_diff.values())), np.array(list(prev_d_diff.values())), atol=convergence_threshold):
            break
        prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    return d_stem, d_diff

# Function to extract ESM features
def extract_esm_features(sequences):
    model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
    model = model.eval()  # Set the model to evaluation mode
    batch_converter = alphabet.get_batch_converter()

    # Prepare data (here we assume sequences_data is a list of protein sequences)
    data = [(str(i), seq) for i, seq in enumerate(sequences)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    
    token_representations = results["representations"][33]
    sequence_representations = token_representations.mean(1).numpy()
    return sequence_representations


# Optional: Fine-tune ESM model on your data
class ProteinDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

def fine_tune_esm_model(sequences, labels, epochs=10, batch_size=32):
    model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
    batch_converter = alphabet.get_batch_converter()
    
    dataset = ProteinDataset(sequences, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        for batch in dataloader:
            sequences, labels = batch
            batch_labels, batch_strs, batch_tokens = batch_converter([(str(i), seq) for i, seq in enumerate(sequences)])

            optimizer.zero_grad()
            results = model(batch_tokens, repr_layers=[33], return_contacts=True)
            token_representations = results["representations"][33]
            sequence_representations = token_representations.mean(1)

            loss = loss_fn(sequence_representations, torch.tensor(labels, dtype=torch.float32))
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    return model

# Load and preprocess biological sequences data
# Assuming sequences_data is a list of protein sequences corresponding to the nodes in the graph
sequences_data = ["MAGVKILKKEKILDYLGELKMHLRGGGSLGGALQRLGEMAKKKMEKGLIHGRGILKLKTP", "MKALLGPWRAVGLLLCALVSCASADGDGALRPPLQGPPAHAARGPHGHPGAPGHPGPPPGLGQP"]  # Replace with actual sequences
X_esm = extract_esm_features(sequences_data)

# Number of samples needed to match X_network
num_samples_needed = len(G.nodes())

# Generate random protein sequences
random_sequences = [generate_random_sequence(20) for _ in range(num_samples_needed)]

# Extract ESM features from random sequences
X_esm = extract_esm_features(random_sequences)

# Placeholder data for training and testing
target_variable = np.random.rand(len(G.nodes()))  # Placeholder for target variable

# Define hyperparameters for the deep learning model
param_grid = {
    'units': 50,
    'optimizer': 'adam',
    'epochs': 10,
    'batch_size': 32
}

print("Shape of adjacency matrix:", adjacency_matrix.shape)
print("Shape of target variable:", target_variable.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(adjacency_matrix, target_variable, test_size=0.2, random_state=42)

# Train deep learning model for feature extraction
model, scaler = train_deep_learning_model(X_train, y_train, param_grid)

# Transform both training and testing data using the fitted scaler
X_combined_train_scaled = scaler.transform(X_train)
X_combined_test_scaled = scaler.transform(X_test)

print("Shape of X_train:", X_train.shape)
print("Shape of X_combined_train:", X_combined_train.shape)
print("Shape of X_combined_train_scaled:", X_combined_train_scaled.shape)


# Calculate proliferation index
pi_scores_train = calculate_pi(G, d_stem, X_network, X_esm, model, scaler)
pi_scores_test = calculate_pi(G, d_diff, X_network, X_esm, model, scaler)

# Placeholder for edge weights based on gene expression data
a = np.random.rand(len(G.nodes()), len(G.nodes()))

# Define initial and final states for the Ricci flow
x_stem = np.random.rand(len(G.nodes()))  # Placeholder for undifferentiated cell state
x_diff = np.random.rand(len(G.nodes()))  # Placeholder for fully differentiated cell state

# Perform discrete Ricci flow iterations
delta_t = 0.1  # Placeholder for time step
num_iterations = 10  # Placeholder for number of iterations
d_stem, d_diff = discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations)

# Calculate proliferation index
pi_scores_stem = calculate_pi(G, d_stem, X_network, X_esm, model, scaler)
pi_scores_diff = calculate_pi(G, d_diff, X_network, X_esm, model, scaler)

# Convert pi_scores_stem and pi_scores_diff to dictionaries for easier manipulation
pi_scores_stem = {node: score for node, score in enumerate(pi_scores_stem)}
pi_scores_diff = {node: score for node, score in enumerate(pi_scores_diff)}

# Categorize cells into groups based on proliferation index
threshold1 = 0.5
threshold2 = 0.3
threshold3 = 0.1

group1 = {node: score for node, score in pi_scores_stem.items() if score > threshold1}
group2 = {node: score for node, score in pi_scores_stem.items() if threshold2 < score <= threshold1}
group3 = {node: score for node, score in pi_scores_stem.items() if threshold3 < score <= threshold2}
group4 = {node: score for node, score in pi_scores_stem.items() if score <= threshold3}

# Print the groups
print("Group 1:", group1)
print("Group 2:", group2)
print("Group 3:", group3)
print("Group 4:", group4)


Shape of adjacency matrix: (10, 10)
Shape of target variable: (10,)
Epoch 1/10


/users/barmanjy/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 950ms/step - loss: 0.3592
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3488
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3380
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3267
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3147
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3020
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2885
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2743
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2592
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2432
Shape of X_train: (8, 10)
Shape of X_combined_train: (10, 1284)
Shape of X_combined_train_scaled: (8, 10)


ValueError: X has 1284 features, but StandardScaler is expecting 10 features as input.

In [2]:
pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 9.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 44.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 24.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━

Note: you may need to restart the kernel to use updated packages.


In [26]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from transformers import BertModel, BertTokenizer
import torch
import random
from sklearn.decomposition import PCA

# Load gene expression data
data = pd.read_csv('/users/barmanjy/Desktop/Persister Cell/GSE1000_series_matrix.txt', delimiter='\t', skiprows=67, header=None)

# Preprocess the data and construct the graph
def preprocess_data(data):
    # Convert non-numeric values to NaN and drop columns with all NaNs
    data = data.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')
    # Calculate correlation matrix
    correlation_matrix = data.corr()
    # Threshold correlations
    threshold = 0.5
    adjacency_matrix = np.where(abs(correlation_matrix) > threshold, 1, 0)
    return adjacency_matrix

adjacency_matrix = preprocess_data(data)
G = nx.from_numpy_array(adjacency_matrix)

# Custom Transformer to extract network features
class NetworkFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, G):
        features = []
        betweenness = nx.betweenness_centrality(G)
        eigenvector = nx.eigenvector_centrality(G)
        for node in G.nodes():
            node_features = [
                G.degree(node),
                nx.clustering(G, node),
                betweenness[node],
                eigenvector[node]
            ]
            features.append(node_features)
        return np.array(features)

# Function to train deep learning model for feature extraction
def train_deep_learning_model(X_train, y_train, param_grid):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    model = Sequential()
    model.add(LSTM(units=param_grid['units'], input_shape=(X_train_scaled.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer=param_grid['optimizer'], loss='mean_squared_error')

    X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
    model.fit(X_train_reshaped, y_train, epochs=param_grid['epochs'], batch_size=param_grid['batch_size'])
    
    return model, scaler

# Function to calculate PI scores using Ricci curvature
def calculate_pi(G, d, X_network, X_bert, model, scaler):
    X_combined = np.concatenate((X_network, X_bert), axis=1)
    X_combined_scaled = scaler.transform(X_combined)
    X_combined_reshaped = np.reshape(X_combined_scaled, (X_combined_scaled.shape[0], X_combined_scaled.shape[1], 1))
    pi_scores = model.predict(X_combined_reshaped)
    return pi_scores.flatten()

# Function to calculate distances based on edge weights
def calculate_distances(G, a, x):
    distances = {}
    for edge in G.edges():
        i, j = edge
        distances[edge] = a[i][j] * x[i] * x[j]
    return distances

# Function to calculate Ricci curvature
def calculate_ricci_curvature(G, a, x):
    ricci_curvature = {}
    for edge in G.edges():
        i, j = edge
        ricci_curvature[edge] = 2 * (a[i][j] - (x[i] + x[j]))
    return ricci_curvature

# Function to update distances in discrete Ricci flow
def update_distances(G, d, ricci_curvature, delta_t):
    updated_distances = {edge: distance + ricci_curvature[edge] * delta_t for edge, distance in d.items()}
    return updated_distances

# Function to perform discrete Ricci flow iterations
def discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations, convergence_threshold=1e-6):
    d_stem = calculate_distances(G, a, x_stem)
    d_diff = calculate_distances(G, a, x_diff)
    ricci_curvature_stem = calculate_ricci_curvature(G, a, x_stem)
    ricci_curvature_diff = calculate_ricci_curvature(G, a, x_diff)
    prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    for _ in range(num_iterations):
        d_stem = update_distances(G, d_stem, ricci_curvature_stem, delta_t)
        d_diff = update_distances(G, d_diff, ricci_curvature_diff, delta_t)
        if np.allclose(np.array(list(d_stem.values())), np.array(list(prev_d_stem.values())), atol=convergence_threshold) and \
           np.allclose(np.array(list(d_diff.values())), np.array(list(prev_d_diff.values())), atol=convergence_threshold):
            break
        prev_d_stem, prev_d_diff = d_stem.copy(), d_diff.copy()
    return d_stem, d_diff

# Function to extract BERT features using ProtBERT
def extract_bert_features(sequences, model, tokenizer, max_length=512):
    features = []
    for seq in sequences:
        inputs = tokenizer(seq, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs)
        sequence_representation = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        features.append(sequence_representation)
    return np.array(features)

# Load ProtBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert')
model = BertModel.from_pretrained('Rostlab/prot_bert')

# Generate random protein sequences (replace this with actual protein sequences)
def generate_random_sequence(length):
    return ''.join(random.choices('ACDEFGHIKLMNPQRSTVWY', k=length))

# Ensure the number of random sequences matches the number of nodes in the graph
num_samples_needed = len(G.nodes())
random_sequences = [generate_random_sequence(20) for _ in range(num_samples_needed)]

# Extract BERT features from protein sequences
X_bert = extract_bert_features(random_sequences, model, tokenizer)

# Reduce the dimensionality of BERT embeddings to match the number of features in X_network
pca = PCA(n_components=4)
X_bert_reduced = pca.fit_transform(X_bert)


# Print shapes of X_network and X_bert before concatenation
print("Shape of X_network before concatenation:", X_network.shape)
print("Shape of X_bert before concatenation:", X_bert.shape)

# Ensure the number of features in X_network and X_bert match
assert X_network.shape[1] == X_bert.shape[1], "Number of features in X_network and X_bert must match."

# Combine network features and BERT features
X_combined = np.concatenate((X_network, X_bert), axis=1)

# Print shape of X_combined after concatenation
print("Shape of X_combined after concatenation:", X_combined.shape)



# Split data into training and testing sets
X_train_network, X_test_network, X_train_bert, X_test_bert, y_train, y_test = train_test_split(
    X_network, X_bert_reduced, target_variable, test_size=0.2, random_state=42
)

# Combine network features and BERT features for training and testing sets
X_train_combined = np.concatenate((X_train_network, X_train_bert), axis=1)
X_test_combined = np.concatenate((X_test_network, X_test_bert), axis=1)

# Define hyperparameters for the deep learning model
param_grid = {
    'units': 50,
    'optimizer': 'adam',
    'epochs': 10,
    'batch_size': 32
}

# Train deep learning model for feature extraction
model, scaler = train_deep_learning_model(X_train_combined, y_train, param_grid)

# Calculate proliferation index for the training and testing sets
pi_scores_train = calculate_pi(G, None, X_train_combined, X_train_bert, model, scaler)
pi_scores_test = calculate_pi(G, None, X_test_combined, X_test_bert, model, scaler)

# Placeholder for edge weights based on gene expression data
a = np.random.rand(len(G.nodes()), len(G.nodes()))

# Define initial and final states for the Ricci flow
x_stem = np.random.rand(len(G.nodes()))
x_diff = np.random.rand(len(G.nodes()))

# Perform discrete Ricci flow iterations
delta_t = 0.1
num_iterations = 10
d_stem, d_diff = discrete_ricci_flow(G, a, x_stem, x_diff, delta_t, num_iterations)

# Calculate proliferation index for the stem and differentiated states
pi_scores_stem = calculate_pi(G, d_stem, X_network, X_bert_reduced, model, scaler)
pi_scores_diff = calculate_pi(G, d_diff, X_network, X_bert_reduced, model, scaler)

# Categorize cells into groups based on proliferation index
threshold1 = 0.5
threshold2 = 0.3
threshold3 = 0.1

group1 = {node: score for node, score in enumerate(pi_scores_stem) if score > threshold1}
group2 = {node: score for node, score in enumerate(pi_scores_stem) if threshold2 < score <= threshold1}
group3 = {node: score for node, score in enumerate(pi_scores_stem) if threshold3 < score <= threshold2}
group4 = {node: score for node, score in enumerate(pi_scores_stem) if score <= threshold3}

# Print the groups
print("Group 1:", group1)
print("Group 2:", group2)
print("Group 3:", group3)
print("Group 4:", group4)



Shape of X_network before concatenation: (10, 4)
Shape of X_bert before concatenation: (10, 1024)


AssertionError: Number of features in X_network and X_bert must match.